In [1]:

import pandas as pd
import tiktoken
from openai.resources.embeddings import Embeddings
import time

In [2]:
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [3]:
df = pd.read_csv("../datacreation/dialogues.csv", sep = '\t')
df = df.dropna()#.head(1000)

In [4]:
df.rename(columns = {'Description':'Question',"Doctor":"Answer"}, inplace = True)

In [5]:
df

,Question,Patient,Answer
0,Will masturbation cause weakness in nerves?,"Hi, may I answer your health queries right now...","Hi, Masturbation does make the nerves weak whe..."
1,Could lack of hair growth be due to masturbation?,hai sir i am 25years old i used to do masturba...,"Hi, Since you said you masturbate frequently e..."
2,What causes seminal leakage?,"Hi, may I answer your health queries right now...","Hi, These are the side effects of the masturba..."
3,What cause sperm discharge while urinating?,i masturbated too much in my life but now by t...,"Hello, Semen discharge during or after urinati..."
4,How to get rid of semen leakage?,"hi,,,sir im 25 years,,of old,im not married an...","Hello, You have visited many doctors and every..."
...,...,...,...
256905,Q. Brother-in-law got a stent for stomach canc...,"Hi doctor, My brother-in-law has stomach cance...",Hi. I understand your concern. For further dou...
256906,Q. Having osteoporosis and spine fracture. How...,"Hi doctor, I have osteoporosis and L3 spine fr...",Hi. I have gone through your message and under...
256907,Q. How long should I take Humira?,"Hi doctor, I am a 55 year old male. I have bee...",Hello. For further information consult a rheum...
256908,Q. Tried withdrawal method and took I-pill. Am...,"Hi doctor, I had unprotected sex and took I-pi...",Hi. For further information consult an obstetr...


In [6]:
df["combined"] = (
    "Question: " + df.Question.str.strip() + "; Patient: " + df.Patient.str.strip()+ "; Answer: " + df.Answer.str.strip()
)
df.head(2)

,Question,Patient,Answer,combined
0,Will masturbation cause weakness in nerves?,"Hi, may I answer your health queries right now...","Hi, Masturbation does make the nerves weak whe...",Question: Will masturbation cause weakness in ...
1,Could lack of hair growth be due to masturbation?,hai sir i am 25years old i used to do masturba...,"Hi, Since you said you masturbate frequently e...",Question: Could lack of hair growth be due to ...


In [7]:
top_n = df.shape[0]

In [8]:
encoding = tiktoken.get_encoding(embedding_encoding)
# omit reviews that are too long to embed
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)
len(df)

256910

In [9]:
df

,Question,Patient,Answer,combined,n_tokens
0,Will masturbation cause weakness in nerves?,"Hi, may I answer your health queries right now...","Hi, Masturbation does make the nerves weak whe...",Question: Will masturbation cause weakness in ...,258
1,Could lack of hair growth be due to masturbation?,hai sir i am 25years old i used to do masturba...,"Hi, Since you said you masturbate frequently e...",Question: Could lack of hair growth be due to ...,194
2,What causes seminal leakage?,"Hi, may I answer your health queries right now...","Hi, These are the side effects of the masturba...",Question: What causes seminal leakage?; Patien...,312
3,What cause sperm discharge while urinating?,i masturbated too much in my life but now by t...,"Hello, Semen discharge during or after urinati...",Question: What cause sperm discharge while uri...,207
4,How to get rid of semen leakage?,"hi,,,sir im 25 years,,of old,im not married an...","Hello, You have visited many doctors and every...",Question: How to get rid of semen leakage?; Pa...,269
...,...,...,...,...,...
256905,Q. Brother-in-law got a stent for stomach canc...,"Hi doctor, My brother-in-law has stomach cance...",Hi. I understand your concern. For further dou...,Question: Q. Brother-in-law got a stent for st...,345
256906,Q. Having osteoporosis and spine fracture. How...,"Hi doctor, I have osteoporosis and L3 spine fr...",Hi. I have gone through your message and under...,Question: Q. Having osteoporosis and spine fra...,140
256907,Q. How long should I take Humira?,"Hi doctor, I am a 55 year old male. I have bee...",Hello. For further information consult a rheum...,Question: Q. How long should I take Humira?; P...,263
256908,Q. Tried withdrawal method and took I-pill. Am...,"Hi doctor, I had unprotected sex and took I-pi...",Hi. For further information consult an obstetr...,Question: Q. Tried withdrawal method and took ...,88


In [10]:
import torch
torch.cuda.is_available()

True

In [11]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

c:\Users\firas\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
#Sentences we want to encode. Example:
sentence = ['This framework generates embeddings for each input sentence']
#Sentences are encoded by calling model.encode()
embedding = model.encode(sentence)

In [13]:
sentence

['This framework generates embeddings for each input sentence']

In [14]:
def get_embeddings(x,transformer='paraphrase-MiniLM-L6-v2'):
    model = SentenceTransformer(transformer)
    #Sentences we want to encode
    sentence =x
    #Sentences are encoded by calling model.encode()
    embedding = model.encode(sentence)
    return embedding

In [1]:

embedding_mod='paraphrase-MiniLM-L6-v2'
#df["embedding"] = df.combined.apply(lambda x: get_embeddings(x, transformer=embedding_mod))

: 

In [16]:
df=df.head(1000)

In [17]:
df["embedding"] = df.Answer.apply(lambda x: get_embeddings(x, transformer=embedding_mod))

C:\Users\firas\AppData\Local\Temp\ipykernel_21756\912830881.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["embedding"] = df.Answer.apply(lambda x: get_embeddings(x, transformer=embedding_mod))


In [18]:
df

,Question,Patient,Answer,combined,n_tokens,embedding
0,Will masturbation cause weakness in nerves?,"Hi, may I answer your health queries right now...","Hi, Masturbation does make the nerves weak whe...",Question: Will masturbation cause weakness in ...,258,"[0.078390285, -0.18259911, 0.26599148, -0.0003..."
1,Could lack of hair growth be due to masturbation?,hai sir i am 25years old i used to do masturba...,"Hi, Since you said you masturbate frequently e...",Question: Could lack of hair growth be due to ...,194,"[0.103233114, 0.17714421, 0.23559776, -0.11525..."
2,What causes seminal leakage?,"Hi, may I answer your health queries right now...","Hi, These are the side effects of the masturba...",Question: What causes seminal leakage?; Patien...,312,"[0.017809762, -0.06392295, 0.3442977, -0.31820..."
3,What cause sperm discharge while urinating?,i masturbated too much in my life but now by t...,"Hello, Semen discharge during or after urinati...",Question: What cause sperm discharge while uri...,207,"[0.16688053, -0.051134076, 0.1188557, -0.12743..."
4,How to get rid of semen leakage?,"hi,,,sir im 25 years,,of old,im not married an...","Hello, You have visited many doctors and every...",Question: How to get rid of semen leakage?; Pa...,269,"[0.01781268, -0.054479472, 0.00820038, -0.0355..."
...,...,...,...,...,...,...
995,What causes feeling of worm movement in scrotum?,I have been feeling some worm like movements i...,"Hello,It may be due to varicocoele. For furthe...",Question: What causes feeling of worm movement...,453,"[-0.14953393, -0.2229069, -0.08538629, 0.04412..."
996,Suggest treatment for masturbation addiction,Is seeing sex vedios is wrongWhile seeing sex ...,"Hi, The discharge from penis is normal.Ways to...",Question: Suggest treatment for masturbation a...,168,"[0.329195, 0.32417792, 0.38875, -0.35095996, -..."
997,What causes night sweating with bleeding?,"Hi, may I answer your health queries right now...","Hi, The bleeding can be due to prostate infect...",Question: What causes night sweating with blee...,159,"[-0.17683402, 0.083951615, 0.33670115, 0.45517..."
998,What is the transparent liquid that is dischar...,"ir, i am 26 year old and u cant believe it, th...","Hi, it is pre-ejaculatory fluid, PED. It’s a n...",Question: What is the transparent liquid that ...,183,"[-0.10605852, -0.0023009975, 0.07539014, -0.20..."


In [19]:
from ast import literal_eval
import numpy as np

In [20]:
df["embedding"] = df.embedding.apply(np.array)  # convert string to numpy array


C:\Users\firas\AppData\Local\Temp\ipykernel_21756\1084155257.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["embedding"] = df.embedding.apply(np.array)  # convert string to numpy array


In [21]:
df.to_pickle("../datacreation/dialogues_embededd.pkl")

In [22]:
df.to_csv("../datacreation/dialogues_embededd.csv", sep = '\t', encoding='utf-8', index=False)